# Find cluster related terms using LIME

## Load the data

In [1]:
root_dir = '../..'
data_dir = 'data'
corpus_dir = 'corpus'
src_dir = 'src'

In [2]:
import os 
import sys

In [3]:
sys.path.append(os.path.join(root_dir, src_dir))

In [4]:
corpus_filename = 'alaska_corpus.json'
corpus_filepath = os.path.join(root_dir, data_dir, corpus_dir, corpus_filename)

In [5]:
chunks_filename = 'alaska_chunks.json'
chunks_filepath = os.path.join(root_dir, data_dir, corpus_dir, chunks_filename)

In [6]:
from training import TrainingCorpus

In [7]:
alaska_corpus = TrainingCorpus()
alaska_corpus.load(corpus_filepath)
alaska_corpus.load_chunks(chunks_filepath)

---

## Load the model

In [8]:
from model import BertModel

Using TensorFlow backend.


In [9]:
model_dir = 'models/alaska_bert'
model_dir_path = os.path.join(root_dir, data_dir, model_dir)

In [10]:
model = BertModel(model_dir_path, batch_size=128, use_cuda=False)

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


---

## Find relevant terms for each cluster label using LIME

### Define utility function for multicore processing

In [11]:
from termfinder import LimeTermFinder

In [12]:
def get_relevant_terms_mp(model, corpus, input_data):
    result_list = []
    
    term_finder = LimeTermFinder(model, corpus)
    
    for label_idx, data_idx in input_data:
        relevant_terms = term_finder.get_relevant_terms(data_idx, label_idx)
        
        if relevant_terms:
            
            for term, weight in relevant_terms.items():
                dict_entry = {'label': corpus.labels[label_idx],
                              'term': term,
                              'weight': weight,
                              'data_id': corpus.docs[data_idx]}
                result_list.append(dict_entry)
    
    return result_list

### Split the data into multiple batches

In [13]:
import numpy as np

In [14]:
label_to_data_idx_dict = model.label_to_data_idx(alaska_corpus)

In [15]:
len(label_to_data_idx_dict.keys())

20

In [15]:
input_data = np.array([(label_idx, data_idx) for label_idx, data_idxs in label_to_data_idx_dict.items()
                       for data_idx in data_idxs])

Get the number of available CPU cores

In [16]:
import psutil

In [17]:
psutil.cpu_count(logical=False)

22

Set the number of parallel jobs

In [18]:
lime_jobs = 12

Compute batches

In [19]:
input_slices = np.array_split(input_data, lime_jobs)

Finally, find relevant terms using `LIME`

In [20]:
from joblib import Parallel, delayed

In [21]:
terms_list_tmp = Parallel(n_jobs=lime_jobs, verbose=10, batch_size=1)(delayed(get_relevant_terms_mp)(model, alaska_corpus, input_batch) for input_batch in input_slices)

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed: 386.3min
[Parallel(n_jobs=12)]: Done   3 out of  12 | elapsed: 398.8min remaining: 1196.5min
[Parallel(n_jobs=12)]: Done   5 out of  12 | elapsed: 400.0min remaining: 560.1min
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed: 405.6min remaining: 289.7min
[Parallel(n_jobs=12)]: Done   9 out of  12 | elapsed: 409.4min remaining: 136.5min
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed: 412.5min finished


Build a DataFrame out of `terms_list_tmp`

In [22]:
df_data = []
for sublist in terms_list_tmp:
    df_data += sublist

In [23]:
df_data[:3]

[{'label': 'ENTITY#44',
  'term': 'nikon_d3200_dslr_camera',
  'weight': 0.8654739472844829,
  'data_id': 0},
 {'label': 'ENTITY#44',
  'term': 'nikon_d3200',
  'weight': 0.8462429120205767,
  'data_id': 1},
 {'label': 'ENTITY#44',
  'term': 'nikon_d3200',
  'weight': 0.7460387218294441,
  'data_id': 2}]

In [24]:
import pandas as pd

In [25]:
relevant_terms_df = pd.DataFrame(df_data)

In [26]:
relevant_terms_df.head()

,label,term,weight,data_id
0,ENTITY#44,nikon_d3200_dslr_camera,0.865474,0
1,ENTITY#44,nikon_d3200,0.846243,1
2,ENTITY#44,nikon_d3200,0.746039,2
3,ENTITY#44,digital_dslr_camera,0.032710,2
4,ENTITY#44,nikon_d3200_digital_dslr_camera,0.819439,3


---

## Save retrieved terms to a file

In [27]:
terms_dir = 'terms'
filename = 'relevant_terms_alaska_bert.csv'
filepath = os.path.join(root_dir, data_dir, terms_dir, filename)

In [28]:
relevant_terms_df.to_csv(filepath, encoding='utf-8', index=False)

---

## Check for pending joblib processes

In [29]:
from multiprocessing import active_children

In [30]:
active_children()

[]

---